# Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
## Load the Dataset
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
## Drop the Null value
df = df.dropna()

In [5]:
df.shape

(18285, 5)

In [6]:
## Get the Independent Features
X = df.drop(columns='label', axis=1)
X.shape

(18285, 4)

In [7]:
## Get the Dependent Feature
y = df['label']
y.shape

(18285,)

In [8]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

Import dependencies for LSTM model

In [9]:
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [10]:
## Vocabulary size
voc_size = 5000

In [11]:
messages = X['title']

In [12]:
len(messages)

18285

In [13]:
messages[1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

Text Preprocessing

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in messages:
    review = re.sub('[^a-zA-Z]', ' ',i).lower().split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

ONE HOT ENCODING

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:5]

[[4357, 1261, 3961, 4835, 3964, 3886, 532, 4911, 1851, 4200],
 [1408, 2465, 4743, 423, 117, 1433, 568],
 [1284, 4557, 2811, 3784],
 [4611, 4669, 951, 735, 226, 919],
 [1599, 117, 2363, 628, 1878, 4583, 117, 4012, 3021, 98]]

Padding for make length of each sentences are same

In [19]:
sent_length=15
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4911 1851 4200]
 [   0    0    0 ...  117 1433  568]
 [   0    0    0 ... 4557 2811 3784]
 ...
 [   0    0    0 ... 4181 1125 3693]
 [   0    0    0 ...  140  336 1443]
 [   0    0    0 ... 3231 3654 4802]]


In [20]:
embedded_docs.shape

(18285, 15)

Creating model

In [21]:
features=40  ## features representation
model=Sequential()

## Add Input/Embedding Layer
model.add(Embedding(voc_size,features,input_length=sent_length))

## Add Hidden/LSTM Layer
model.add(Bidirectional(LSTM(100)))  ## Taking 1 LSTM Layer with 100 Neurons

## Add Output Layer
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
## Convert into Array
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [23]:
X_final.shape,y_final.shape

((18285, 15), (18285,))

Splitng into train and test data

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [25]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=100)

Epoch 1/10
123/123 [==============================] - 14s 61ms/step - loss: 0.3252 - accuracy: 0.8359 - val_loss: 0.2137 - val_accuracy: 0.9112
Epoch 2/10
123/123 [==============================] - 9s 73ms/step - loss: 0.1384 - accuracy: 0.9469 - val_loss: 0.2074 - val_accuracy: 0.9188
Epoch 3/10
123/123 [==============================] - 6s 46ms/step - loss: 0.0946 - accuracy: 0.9643 - val_loss: 0.2431 - val_accuracy: 0.9171
Epoch 4/10
123/123 [==============================] - 6s 47ms/step - loss: 0.0644 - accuracy: 0.9761 - val_loss: 0.3057 - val_accuracy: 0.9125
Epoch 5/10
123/123 [==============================] - 6s 47ms/step - loss: 0.0449 - accuracy: 0.9843 - val_loss: 0.3401 - val_accuracy: 0.9158
Epoch 6/10
123/123 [==============================] - 6s 47ms/step - loss: 0.0300 - accuracy: 0.9892 - val_loss: 0.3558 - val_accuracy: 0.9059
Epoch 7/10
123/123 [==============================] - 6s 48ms/step - loss: 0.0187 - accuracy: 0.9945 - val_loss: 0.4387 - val_accuracy: 0.913

# NOTICE PROBLEM

Model Training Accuracy is 100% & Testing accurace is 90%. So, it is overfitting situation.
Now, We need to regulized model with add droput layer.

Regulized the model 

In [31]:
features=30  ##features representation
model=Sequential()

## Add Input/Embedding Layer
model.add(Embedding(voc_size,features,input_length=sent_length))
model.add(Dropout(0.7))

## Add Hidden/LSTM Layer
model.add(Bidirectional(LSTM(100)))  ## Taking 1 LSTM Layer with 100 Neurons
model.add(Dropout(0.7))

## Add Output Layer
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 15, 30)            150000    
                                                                 
 dropout_2 (Dropout)         (None, 15, 30)            0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              104800    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 255,001
Trainable params: 255,001
Non-trainable params: 0
________________________________________________

In [32]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=100)

Epoch 1/10
123/123 [==============================] - 11s 51ms/step - loss: 0.3609 - accuracy: 0.8174 - val_loss: 0.2165 - val_accuracy: 0.9070
Epoch 2/10
123/123 [==============================] - 5s 39ms/step - loss: 0.1994 - accuracy: 0.9200 - val_loss: 0.2052 - val_accuracy: 0.9140
Epoch 3/10
123/123 [==============================] - 5s 43ms/step - loss: 0.1609 - accuracy: 0.9366 - val_loss: 0.1972 - val_accuracy: 0.9168
Epoch 4/10
123/123 [==============================] - 5s 40ms/step - loss: 0.1370 - accuracy: 0.9476 - val_loss: 0.1946 - val_accuracy: 0.9193
Epoch 5/10
123/123 [==============================] - 5s 43ms/step - loss: 0.1216 - accuracy: 0.9538 - val_loss: 0.2316 - val_accuracy: 0.9173
Epoch 6/10
123/123 [==============================] - 6s 51ms/step - loss: 0.1106 - accuracy: 0.9564 - val_loss: 0.2348 - val_accuracy: 0.9175
Epoch 7/10
123/123 [==============================] - 5s 42ms/step - loss: 0.1005 - accuracy: 0.9613 - val_loss: 0.2250 - val_accuracy: 0.918

Performance Metrics And Accuracy

In [33]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0)
y_pred

189/189 [==============================] - 2s 7ms/step


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3114,  305],
       [ 225, 2391]], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9121789560894781

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

